Goal of this notebook is to test wide range of potential models.

1. Lazypredict
-> choose promising models
2. GridSearch on selection of promising models
3. Summary of hot candidates including metric, high influencing variables, pca of cluster, etc.


Open todo's-> (maybe other notebook)
- hyperparameter optimization
- dimension reduction

# Lazy Predict

We'll use LazyRegressor as we're dealing with a supervised learning regression problem and want to check potential models for our usecase

Select models from LazyRegressor as processing failed for some in previous sessions.

In [2]:
from copy import deepcopy
from datetime import datetime

import joblib
import pandas as pd
from config import OUTPUT_DIR, TRAIN_TEST_SPLIT_ELECTRIC_FILE
from lazypredict.Supervised import REGRESSORS, LazyRegressor

In [3]:
current_date = datetime.now().strftime('%Y_%m_%d')

In [4]:
current_date = datetime.now().strftime('%Y_%m_%d')
X_train, X_test, y_train, y_test = joblib.load(TRAIN_TEST_SPLIT_ELECTRIC_FILE)

In [5]:
# explicit selection through mannual list (based on lazypredict.Supervised.REGRESSION)

# import sklearn
# import xgboost
# import lightgbm

# GaussianProcessRegressor model failed to execute
# KernelRidge model failed to execute
# LazyRegressor stuck at 62% (26/42) -> 25 MLPRegressor || 26 NuSVR?

# model_selection = [
#     ('AdaBoostRegressor', sklearn.ensemble._weight_boosting.AdaBoostRegressor),
#     ('BaggingRegressor', sklearn.ensemble._bagging.BaggingRegressor),
#     ('BayesianRidge', sklearn.linear_model._bayes.BayesianRidge),
#     ('DecisionTreeRegressor', sklearn.tree._classes.DecisionTreeRegressor),
#     ('DummyRegressor', sklearn.dummy.DummyRegressor),
#     ('ElasticNet', sklearn.linear_model._coordinate_descent.ElasticNet),
#     ('ElasticNetCV', sklearn.linear_model._coordinate_descent.ElasticNetCV),
#     ('ExtraTreeRegressor', sklearn.tree._classes.ExtraTreeRegressor),
#     ('ExtraTreesRegressor', sklearn.ensemble._forest.ExtraTreesRegressor),
#     ('GammaRegressor', sklearn.linear_model._glm.glm.GammaRegressor),
#     ('GaussianProcessRegressor', sklearn.gaussian_process._gpr.GaussianProcessRegressor),
#     ('GradientBoostingRegressor', sklearn.ensemble._gb.GradientBoostingRegressor),
#     ('HistGradientBoostingRegressor', sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingRegressor),
#     ('HuberRegressor', sklearn.linear_model._huber.HuberRegressor),
#     ('KNeighborsRegressor', sklearn.neighbors._regression.KNeighborsRegressor),
#     ('KernelRidge', sklearn.kernel_ridge.KernelRidge),
#     ('Lars', sklearn.linear_model._least_angle.Lars),
#     ('LarsCV', sklearn.linear_model._least_angle.LarsCV),
#     ('Lasso', sklearn.linear_model._coordinate_descent.Lasso),
#     ('LassoCV', sklearn.linear_model._coordinate_descent.LassoCV),
#     ('LassoLars', sklearn.linear_model._least_angle.LassoLars),
#     ('LassoLarsCV', sklearn.linear_model._least_angle.LassoLarsCV),
#     ('LassoLarsIC', sklearn.linear_model._least_angle.LassoLarsIC),
#     ('LinearRegression', sklearn.linear_model._base.LinearRegression),
#     ('LinearSVR', sklearn.svm._classes.LinearSVR),
#     ('MLPRegressor', sklearn.neural_network._multilayer_perceptron.MLPRegressor),
#     ('NuSVR', sklearn.svm._classes.NuSVR),
#     ('OrthogonalMatchingPursuit', sklearn.linear_model._omp.OrthogonalMatchingPursuit),
#     ('OrthogonalMatchingPursuitCV', sklearn.linear_model._omp.OrthogonalMatchingPursuitCV),
#     ('PassiveAggressiveRegressor', sklearn.linear_model._passive_aggressive.PassiveAggressiveRegressor),
#     ('PoissonRegressor', sklearn.linear_model._glm.glm.PoissonRegressor),
#     ('QuantileRegressor', sklearn.linear_model._quantile.QuantileRegressor),
#     ('RANSACRegressor', sklearn.linear_model._ransac.RANSACRegressor),
#     ('RandomForestRegressor', sklearn.ensemble._forest.RandomForestRegressor),
#     ('Ridge', sklearn.linear_model._ridge.Ridge),
#     ('RidgeCV', sklearn.linear_model._ridge.RidgeCV),
#     ('SGDRegressor', sklearn.linear_model._stochastic_gradient.SGDRegressor),
#     ('SVR', sklearn.svm._classes.SVR),
#     ('TransformedTargetRegressor', sklearn.compose._target.TransformedTargetRegressor),
#     ('TweedieRegressor', sklearn.linear_model._glm.glm.TweedieRegressor),
#     ('XGBRegressor', xgboost.sklearn.XGBRegressor),
#     ('LGBMRegressor', lightgbm.sklearn.LGBMRegressor)
# ]

# selection through ignore
model_selection = REGRESSORS
"""
----
Removed due to failed executions:
----
GaussianProcessRegressor model failed to execute
KernelRidge model failed to execute
LazyRegressor stuck at 62% (26/42) -> 25 MLPRegressor || 26 NuSVR?

----
Removed due to high compute time:
----
LassoCV
QuantileRegressor
RANSACRegressor

----
Compute error messages
----
LarsCV model failed to execute
Unable to allocate 4.07 GiB for an array with shape (1824768, 299) and data type float64

LassoLarsCV model failed to execute
Unable to allocate 4.07 GiB for an array with shape (1824768, 299) and data type float64
    
LassoLarsIC model failed to execute
Unable to allocate 7.04 GiB for an array with shape (2280960, 414) and data type float64
    
RidgeCV model failed to execute
Unable to allocate 5.10 GiB for an array with shape (2280960, 300) and data type float64

SVR -> stuck
"""
models_to_ignore = list([
    "GaussianProcessRegressor",
     "KernelRidge",
     "LassoCV",
     "NuSVR",
     "MLPRegressor",
     "QuantileRegressor",
     "RANSACRegressor",
     "LarsCV",
     "LassoLarsCV",
     "LassoLarsIC",
     "RidgeCV",
     "SVR"
])

Use chunk approach

In [6]:
class LazyPredictChunk:
    def __init__(self, models=None, predictions=None, regressors="all"):
        self.models = models
        self.predictions = predictions
        self.regressors = regressors

In [7]:
chunk_size = 1
lazy_predict_chunks = []

# split regressors into chunks
i = 0
chunk_buffer = []
for model, model_class in model_selection:
    # ignore models that are in ignore list
    if model in models_to_ignore:
        i+=1
        continue

    # append if chunk_buffer below size limit
    if len(chunk_buffer) < chunk_size:
        chunk_buffer.append((model, model_class))

    # chunk size reached or last iteration step reached
    if len(chunk_buffer) == chunk_size or (i+1) == len(REGRESSORS):
        lazy_predict_chunks.append(LazyPredictChunk(regressors=deepcopy(chunk_buffer)))
        # empty chunk_buffer
        chunk_buffer = []

    i+=1

In [8]:
print("Executing LazyRegressor through: ", len(lazy_predict_chunks), "chunks")

# save compute if executed without predictions
predictions_flag = False

for chunk in lazy_predict_chunks:
    print("Processings the following regressors in this chunk: ", chunk.regressors)
    reg = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None, regressors=chunk.regressors, predictions=predictions_flag)
    models,predictions = reg.fit(X_train, X_test, y_train, y_test)
    # models,predictions = reg.fit(X_train[:500], X_test[:500], y_train[:500], y_test[:500])

    # save trained models with joblib
    for model_name, model in reg.models.items():
        joblib.dump(model, f"{OUTPUT_DIR}models/{current_date}-lazy-{model_name}.pkl")

    # store results into LazyPredictChunk variables
    chunk.models = deepcopy(models) # deepcopy to not have weird references
    if predictions_flag:
        chunk.predictions = deepcopy(predictions)

Executing LazyRegressor through:  30 chunks
Processings the following regressors in this chunk:  [('AdaBoostRegressor', <class 'sklearn.ensemble._weight_boosting.AdaBoostRegressor'>)]
'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 1/1 [04:01<00:00, 241.87s/it]


Processings the following regressors in this chunk:  [('BaggingRegressor', <class 'sklearn.ensemble._bagging.BaggingRegressor'>)]
'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 1/1 [00:58<00:00, 58.77s/it]


Processings the following regressors in this chunk:  [('BayesianRidge', <class 'sklearn.linear_model._bayes.BayesianRidge'>)]
'tuple' object has no attribute '__name__'
Invalid Regressor(s)


...

100%|██████████| 1/1 [00:13<00:00, 13.90s/it]


Processings the following regressors in this chunk:  [('LGBMRegressor', <class 'lightgbm.sklearn.LGBMRegressor'>)]
'tuple' object has no attribute '__name__'
Invalid Regressor(s)


  0%|          | 0/1 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.113993 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1827
[LightGBM] [Info] Number of data points in the train set: 304128, number of used features: 297
[LightGBM] [Info] Start training from score 168.054997


100%|██████████| 1/1 [00:11<00:00, 11.90s/it]


In [9]:
# build combined results
all_models = pd.DataFrame()
all_predictions = pd.DataFrame()

for chunk in lazy_predict_chunks:
    all_models = pd.concat([all_models, chunk.models])
    if predictions_flag:
        all_predictions = pd.concat([all_predictions, chunk.predictions])

In [10]:
output_file = OUTPUT_DIR + current_date + "-lazy_models.csv"
all_models.to_csv(output_file)
all_models

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
AdaBoostRegressor,0.31,0.31,20.27,241.86
BaggingRegressor,0.98,0.98,3.30,58.77
BayesianRidge,0.87,0.87,8.93,20.62
DecisionTreeRegressor,0.98,0.98,3.67,15.06
DummyRegressor,-0.00,-0.00,24.43,8.86
ElasticNet,0.75,0.75,12.17,12.83
ElasticNetCV,0.86,0.86,9.08,56.70
ExtraTreeRegressor,0.98,0.98,3.33,17.29
ExtraTreesRegressor,0.98,0.98,3.02,675.06


In [11]:
if predictions_flag:
    output_file = OUTPUT_DIR + current_date + "-lazy_predictions.csv"
    all_predictions.to_csv(output_file)
    all_predictions

In [12]:
print("Predictions ran on X_train of shape: ", X_train.shape)

Predictions ran on X_train of shape:  (304128, 299)


In [13]:
all_models.columns

Index(['Adjusted R-Squared', 'R-Squared', 'RMSE', 'Time Taken'], dtype='object')

Models sorted by R-Squared

In [ ]:
all_models.sort_values(by=['R-Squared'], ascending=False)

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
ExtraTreesRegressor,0.98,0.98,3.02,675.06
RandomForestRegressor,0.98,0.98,3.19,323.68
BaggingRegressor,0.98,0.98,3.30,58.77
ExtraTreeRegressor,0.98,0.98,3.33,17.29
DecisionTreeRegressor,0.98,0.98,3.67,15.06
KNeighborsRegressor,0.98,0.98,3.84,1244.26
LGBMRegressor,0.96,0.96,4.75,11.89
HistGradientBoostingRegressor,0.96,0.96,5.08,24.99
GradientBoostingRegressor,0.89,0.89,7.95,152.40


Models sorted by Time Taken

In [ ]:
all_models.sort_values(by=['Time Taken'], ascending=False)

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
KNeighborsRegressor,0.98,0.98,3.84,1244.26
ExtraTreesRegressor,0.98,0.98,3.02,675.06
RandomForestRegressor,0.98,0.98,3.19,323.68
LinearSVR,0.84,0.84,9.66,284.12
AdaBoostRegressor,0.31,0.31,20.27,241.86
GradientBoostingRegressor,0.89,0.89,7.95,152.40
HuberRegressor,0.85,0.85,9.56,105.49
BaggingRegressor,0.98,0.98,3.30,58.77
ElasticNetCV,0.86,0.86,9.08,56.70


In [19]:
joblib.dump(lazy_predict_chunks, f"{OUTPUT_DIR}models/{current_date}-lazy_predict_chunks.pkl")

['files/output/models/2025_03_10-lazy_predict_chunks.pkl']